In [ ]:
# code with graphics

In [1]:
from copy import deepcopy
from colorama import Fore, Back, Style

DIRECTIONS = {"U": [-1, 0], "D": [1, 0], "L": [0, -1], "R": [0, 1]}
END = [[1, 2, 3], [4, 5, 6], [7, 8, 0]]

# unicode
left_down_angle = '\u2514'
right_down_angle = '\u2518'
right_up_angle = '\u2510'
left_up_angle = '\u250C'

middle_junction = '\u253C'
top_junction = '\u252C'
bottom_junction = '\u2534'
right_junction = '\u2524'
left_junction = '\u251C'

bar = Style.BRIGHT + Fore.CYAN + '\u2502' + Fore.RESET + Style.RESET_ALL
dash = '\u2500'

first_line = Style.BRIGHT + Fore.CYAN + left_up_angle + dash + dash + dash + top_junction + dash + dash + dash + top_junction + dash + dash + dash + right_up_angle + Fore.RESET + Style.RESET_ALL
middle_line = Style.BRIGHT + Fore.CYAN + left_junction + dash + dash + dash + middle_junction + dash + dash + dash + middle_junction + dash + dash + dash + right_junction + Fore.RESET + Style.RESET_ALL
last_line = Style.BRIGHT + Fore.CYAN + left_down_angle + dash + dash + dash + bottom_junction + dash + dash + dash + bottom_junction + dash + dash + dash + right_down_angle + Fore.RESET + Style.RESET_ALL


def print_puzzle(array):
    print(first_line)
    for a in range(len(array)):
        for i in array[a]:
            if i == 0:
                print(bar, Back.RED + ' ' + Back.RESET, end=' ')
            else:
                print(bar, i, end=' ')
        print(bar)
        if a == 2:
            print(last_line)
        else:
            print(middle_line)


class Node:
    def __init__(self, current_node, previous_node, g, h, dir):
        self.current_node = current_node
        self.previous_node = previous_node
        self.g = g
        self.h = h
        self.dir = dir

    def f(self):
        return self.g + self.h


def get_pos(current_state, element):
    for row in range(len(current_state)):
        if element in current_state[row]:
            return (row, current_state[row].index(element))


def euclidianCost(current_state):
    cost = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[0])):
            pos = get_pos(END, current_state[row][col])
            cost += abs(row - pos[0]) + abs(col - pos[1])
    return cost


def getAdjNode(node):
    listNode = []
    emptyPos = get_pos(node.current_node, 0)

    for dir in DIRECTIONS.keys():
        newPos = (emptyPos[0] + DIRECTIONS[dir][0], emptyPos[1] + DIRECTIONS[dir][1])
        if 0 <= newPos[0] < len(node.current_node) and 0 <= newPos[1] < len(node.current_node[0]):
            newState = deepcopy(node.current_node)
            newState[emptyPos[0]][emptyPos[1]] = node.current_node[newPos[0]][newPos[1]]
            newState[newPos[0]][newPos[1]] = 0
            # listNode += [Node(newState, node.current_node, node.g + 1, euclidianCost(newState), dir)]
            listNode.append(Node(newState, node.current_node, node.g + 1, euclidianCost(newState), dir))

    return listNode


def getBestNode(openSet):
    firstIter = True

    for node in openSet.values():
        if firstIter or node.f() < bestF:
            firstIter = False
            bestNode = node
            bestF = bestNode.f()
    return bestNode


def buildPath(closedSet):
    node = closedSet[str(END)]
    branch = list()

    while node.dir:
        branch.append({
            'dir': node.dir,
            'node': node.current_node
        })
        node = closedSet[str(node.previous_node)]
    branch.append({
        'dir': '',
        'node': node.current_node
    })
    branch.reverse()

    return branch


def main(puzzle):
    open_set = {str(puzzle): Node(puzzle, puzzle, 0, euclidianCost(puzzle), "")}
    closed_set = {}

    while True:
        test_node = getBestNode(open_set)
        closed_set[str(test_node.current_node)] = test_node

        if test_node.current_node == END:
            return buildPath(closed_set)

        adj_node = getAdjNode(test_node)
        for node in adj_node:
            if str(node.current_node) in closed_set.keys() or str(node.current_node) in open_set.keys() and open_set[
                str(node.current_node)].f() < node.f():
                continue
            open_set[str(node.current_node)] = node

        del open_set[str(test_node.current_node)]


if __name__ == '__main__':
    br = main([[6, 2, 8],
               [4, 7, 1],
               [0, 3, 5]])

    print('total steps : ', len(br) - 1)
    print()
    print(dash + dash + right_junction, "INPUT", left_junction + dash + dash)
    for b in br:
        if b['dir'] != '':
            letter = ''
            if b['dir'] == 'U':
                letter = 'UP'
            elif b['dir'] == 'R':
                letter = "RIGHT"
            elif b['dir'] == 'L':
                letter = 'LEFT'
            elif b['dir'] == 'D':
                letter = 'DOWN'
            print(dash + dash + right_junction, letter, left_junction + dash + dash)
        print_puzzle(b['node'])
        print()

    print(dash + dash + right_junction, 'ABOVE IS THE OUTPUT', left_junction + dash + dash)


total steps :  24

──┤ INPUT ├──
┌───┬───┬───┐
│ 6 │ 2 │ 8 │
├───┼───┼───┤
│ 4 │ 7 │ 1 │
├───┼───┼───┤
│   │ 3 │ 5 │
└───┴───┴───┘

──┤ RIGHT ├──
┌───┬───┬───┐
│ 6 │ 2 │ 8 │
├───┼───┼───┤
│ 4 │ 7 │ 1 │
├───┼───┼───┤
│ 3 │   │ 5 │
└───┴───┴───┘

──┤ UP ├──
┌───┬───┬───┐
│ 6 │ 2 │ 8 │
├───┼───┼───┤
│ 4 │   │ 1 │
├───┼───┼───┤
│ 3 │ 7 │ 5 │
└───┴───┴───┘

──┤ RIGHT ├──
┌───┬───┬───┐
│ 6 │ 2 │ 8 │
├───┼───┼───┤
│ 4 │ 1 │   │
├───┼───┼───┤
│ 3 │ 7 │ 5 │
└───┴───┴───┘

──┤ UP ├──
┌───┬───┬───┐
│ 6 │ 2 │   │
├───┼───┼───┤
│ 4 │ 1 │ 8 │
├───┼───┼───┤
│ 3 │ 7 │ 5 │
└───┴───┴───┘

──┤ LEFT ├──
┌───┬───┬───┐
│ 6 │   │ 2 │
├───┼───┼───┤
│ 4 │ 1 │ 8 │
├───┼───┼───┤
│ 3 │ 7 │ 5 │
└───┴───┴───┘

──┤ LEFT ├──
┌───┬───┬───┐
│   │ 6 │ 2 │
├───┼───┼───┤
│ 4 │ 1 │ 8 │
├───┼───┼───┤
│ 3 │ 7 │ 5 │
└───┴───┴───┘

──┤ DOWN ├──
┌───┬───┬───┐
│ 4 │ 6 │ 2 │
├───┼───┼───┤
│   │ 1 │ 8 │
├───┼───┼───┤
│ 3 │ 7 │ 5 │
└───┴───┴───┘

──┤ DOWN ├──
┌───┬───┬───┐
│ 4 │ 6 │ 2 │
├───┼───┼───┤
│ 3 │ 1 │ 8 │
├───┼───┼───┤
│  

In [6]:
class eightpuzzleproblem:
    def __inti__(self, i_state,g_state):
        self.i_state=i_state
        self.g_state=g_state
        self.step_cost=1
    def action(self,state):
        possible_action={'up','down','right','left'}
        blank_id=state.index(0)
        if(bank_id%3==0):
            possible_action.remove('left')
        if(bank_id%3==1):
            possible_action.remove('right')
        if(bank_id%3==2):
            possible_action.remove('up')
        if(bank_id%3==3):
            possible_action.remove('down')
        return possible_action
    def result(self,state,action):
        blank_id=state.index(0)
        if (action=='up'):
            state[blank_id],state[blank_id-3]=state[blank_id-3],state[blank_id]
        if (action=='down'):
            state[blank_id],state[blank_id+3]=state[blank_id+3],state[blank_id]
        if (action=='left'):
            state[blank_id],state[blank_id-1]=state[blank_id-1],state[blank_id]
        if (action=='right'):
            state[blank_id],state[blank_id+1]=state[blank_id+1],state[blank_id]

        return state
    def step_cost(self,cur_state,nx_state):
        return self.step_cost
    def path_cost(self,cur_state,cur_cost,nx_state):
        return cur_cost+ step_cost(cur_state)
    def goal_test(self,state):
        return (state==self.goal)

In [8]:
class node:
    def __init__(self,state,parent,action,cost):
        self.state=state
        self.parent=parent
        self.action=action
        if(parent):
            self.depth=depth
        else:
            self.depth=0
    def child_node(self,action,problem):
        return node(problem.result(self.state,action),self,action,self.cost+problem.step_cost(self.state,problem.result(self.state,action))
    def expand(self,possible_action,problem):
        list_succ=[]
        for action in possible_action:
            list_succ.append(self.child_node(action,problem))
        return list_succ
    def solution(self):
        node,solution=self,[]
        while (node.parent):
            solution.append(node.action)
            node=node.parent
        return list(reversed(solution))


SyntaxError: invalid syntax (<ipython-input-8-9a0ae7ebbaf2>, line 12)

In [ ]:
class eightpuzzlesol:
    def iterative_deepening_search():
        pass
class optimalsolutionchecker:
    pass
